## Coding Exercise #0308

Predicción de supervivencia: El objetivo principal es construir un modelo de aprendizaje automático que pueda predecir si un pasajero del Titanic sobrevivió o no, basándose en varias características como clase, sexo, edad, etc.

Optimización de modelo: El notebook se centra en cómo optimizar un árbol de decisión, que es un tipo de modelo de clasificación. Utiliza la técnica de búsqueda en cuadrícula (Grid Search) para encontrar los mejores hiperparámetros para el árbol de decisión.

Comparación de rendimiento: Se busca comparar el rendimiento de un árbol de decisión con parámetros optimizados frente a uno con parámetros por defecto, para demostrar la importancia de la optimización de hiperparámetros.

Práctica con scikit-learn: El notebook sirve como un ejercicio práctico para utilizar varias funcionalidades de scikit-learn, como división de datos, árboles de decisión y búsqueda en cuadrícula.

Evaluación de modelos: Se utiliza la métrica de precisión (accuracy) para evaluar qué tan bien el modelo puede predecir la supervivencia de los pasajeros.

### 1. Classification with Tree:

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

#### 1.1. Read in data:

In [41]:
df = pd.read_csv('./76._data_titanic_2.csv', header='infer')

In [8]:
df.shape

(889, 21)

In [10]:
df.head(3)

,Embarked_Q,Embarked_S,Sex_male,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,SibSp_1,...,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Pclass_2,Pclass_3,"Age_(21.0, 30.0]","Age_(30.0, 35.0]","Age_(35.0, 80.0]",Survived
0,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1


In [12]:
X = df.drop(columns=['Survived'])
Y = df.Survived  # Queremos saber los supervivientes

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

#### 1.2. Tree hyperparameter optimization:

In [23]:
# P: Establece los valores de depth_grid en [1,21], min_samples_leaf_grid en [10,31] y max_leaf_nodes_grid en [2,21] 
# utilizando https://numpy.org/doc/stable/reference/generated/numpy.arange.html
depth_grid = np.arange(1, 21)
min_samples_leaf_grid = np.arange(10, 31)
max_leaf_nodes_grid = np.arange(2, 21)
parameters = {'max_depth':depth_grid, 'min_samples_leaf':min_samples_leaf_grid, 'max_leaf_nodes':max_leaf_nodes_grid}

In [25]:
# P: Instanciar la función de búsqueda en cuadrícula (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
# para optimizar los hiperparámetros (proporcionando la variable "parameters" anterior) utilizando un clasificador de árbol de decisión como estimador, 
# con una validación cruzada de 10 veces y -1 como el número de trabajos para ejecutar en paralelo.

gridCV = GridSearchCV(estimator=DecisionTreeClassifier(), 
                      param_grid=parameters, 
                      cv=10, 
                      n_jobs=-1)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']
best_min_samples_leaf = gridCV.best_params_['min_samples_leaf']
best_max_leaf_nodes = gridCV.best_params_['max_leaf_nodes']

In [31]:
print("Tree best depth : " + str(best_depth))
print("Tree best min_samples_leaf : " + str(best_min_samples_leaf))
print("Tree best max_leaf_nodes : " + str(best_max_leaf_nodes))

Tree best depth : 3
Tree best min_samples_leaf : 23
Tree best max_leaf_nodes : 6


In [33]:
# P: Instanciar un clasificador de árbol de decisión con los mejores parámetros obtenidos anteriormente. 
DTC_best = DecisionTreeClassifier(max_depth=best_depth, 
                                  min_samples_leaf=best_min_samples_leaf, 
                                  max_leaf_nodes=best_max_leaf_nodes)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Tree best accuracy : 0.82


0.82 Esta es la precisión del árbol de decisión con los mejores hiperparámetros encontrados mediante la búsqueda en cuadrícula. Significa que el modelo optimizado predice correctamente si un pasajero sobrevivió o no en el 82% de los casos en el conjunto de prueba

In [35]:
# P: ¿Cuál sería la precisión (ACC) de un árbol de decisión con los parámetros predeterminados? ¿Cuál es la mejora que obtenemos con una búsqueda en cuadrícula de los mejores hiperparámetros?

DTC_default = DecisionTreeClassifier()
DTC_default.fit(X_train, Y_train)
Y_pred_default = DTC_default.predict(X_test)
default_accuracy = np.round(metrics.accuracy_score(Y_test, Y_pred_default), 3)
print("Tree default accuracy : " + str(default_accuracy))

improvement = np.round(metrics.accuracy_score(Y_test, Y_pred) - default_accuracy, 3)
print("Improvement with grid search: " + str(improvement))

Tree default accuracy : 0.787
Improvement with grid search: 0.033


0.787
Esta es la precisión del árbol de decisión con los parámetros por defecto. Indica que el modelo sin optimizar predice correctamente la supervivencia en el 78.7% de los casos en el conjunto de prueba.